In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import gc

# ==========================================
# CONFIGURATION
# ==========================================
CFG = {
    'input_dim': 1280,      
    'num_classes': 1500,    
    'batch_size': 32,      
    'epochs': 10,
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu")
}
print(f"Using device: {CFG['device']}")

# ==========================================
# 1. LOAD DATA
# ==========================================
print(">>> Loading Data...")
train_emb = np.load("/kaggle/input/emb-models-ttt/train_embeds.npy").astype(np.float32)
train_ids = np.load("/kaggle/input/emb-models-ttt/train_ids.npy")
test_emb = np.load("/kaggle/input/emb-models-ttt/test_embeds.npy").astype(np.float32)
test_ids = np.load("/kaggle/input/emb-models-ttt/test_ids.npy")

train_terms = pd.read_csv("/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv", sep="\t")
term_counts = train_terms['term'].value_counts()
top_terms = term_counts.index[:CFG['num_classes']].tolist()
term_to_idx = {term: i for i, term in enumerate(top_terms)}

# Prepare Labels
print(">>> Building Target Matrix...")
train_id_set = set(train_ids)
filtered_terms = train_terms[train_terms['term'].isin(top_terms) & train_terms['EntryID'].isin(train_id_set)]
id_to_index = {pid: i for i, pid in enumerate(train_ids)}
labels = np.zeros((len(train_ids), CFG['num_classes']), dtype=np.float32)

for pid, term in tqdm(zip(filtered_terms['EntryID'], filtered_terms['term']), total=len(filtered_terms)):
    if pid in id_to_index:
        labels[id_to_index[pid], term_to_idx[term]] = 1.0

del train_terms, filtered_terms, term_counts
gc.collect()

# ==========================================
# 2. DEFINE THE "TRINITY" ARCHITECTURES
# ==========================================

# Model A: Simple MLP (The "Classic" - often scores 0.19+)
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

# Model B: ResMLP (The "Deep" one)
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.BatchNorm1d(in_features),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
    def forward(self, x): return x + self.block(x)

class ResMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.input_layer = nn.Sequential(nn.Linear(input_dim, 512), nn.BatchNorm1d(512), nn.ReLU())
        self.res_blocks = nn.Sequential(ResidualBlock(512), ResidualBlock(512))
        self.output_layer = nn.Linear(512, num_classes)
    def forward(self, x):
        return self.output_layer(self.res_blocks(self.input_layer(x)))

# Model C: Wide Bottleneck (The "Capacity" one)
class WideNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), # Wide layer
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, num_classes)
        )
    def forward(self, x): return self.net(x)

# ==========================================
# 3. TRAINING LOOP
# ==========================================
class ProteinDataset(Dataset):
    def __init__(self, embeddings, targets=None):
        self.embeddings = embeddings
        self.targets = targets
    def __len__(self): return len(self.embeddings)
    def __getitem__(self, idx):
        x = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        if self.targets is not None:
            return x, torch.tensor(self.targets[idx], dtype=torch.float32)
        return x

# Train List
models_to_train = [
    ("SimpleMLP", SimpleMLP(CFG['input_dim'], CFG['num_classes'])),
    ("ResMLP", ResMLP(CFG['input_dim'], CFG['num_classes'])),
    ("WideNet", WideNet(CFG['input_dim'], CFG['num_classes']))
]

ensemble_preds = np.zeros((len(test_ids), CFG['num_classes']), dtype=np.float32)

for name, model in models_to_train:
    print(f"\n>>> Training Architecture: {name}...")
    
    train_loader = DataLoader(ProteinDataset(train_emb, labels), batch_size=CFG['batch_size'], shuffle=True, num_workers=2)
    model = model.to(CFG['device'])
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(CFG['epochs']):
        model.train()
        for batch_x, batch_y in tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False):
            batch_x, batch_y = batch_x.to(CFG['device']), batch_y.to(CFG['device'])
            optimizer.zero_grad()
            loss = criterion(model(batch_x), batch_y)
            loss.backward()
            optimizer.step()

    print(f">>> Predicting with {name}...")
    test_loader = DataLoader(ProteinDataset(test_emb), batch_size=CFG['batch_size'], shuffle=False, num_workers=2)
    model.eval()
    temp_preds = []
    with torch.no_grad():
        for batch_x in tqdm(test_loader, desc="Inference"):
            probs = torch.sigmoid(model(batch_x.to(CFG['device']))).cpu().numpy()
            temp_preds.append(probs)
    
    ensemble_preds += np.concatenate(temp_preds)
    del model, optimizer, train_loader, test_loader, temp_preds
    torch.cuda.empty_cache()
    gc.collect()

# Average
ensemble_preds /= len(models_to_train)
print(">>> Diverse Ensemble Averaging Complete.")

# ==========================================
# 4. HIERARCHY PROPAGATION
# ==========================================
print(">>> Parsing Graph...")
parents = {}
with open("/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo", 'r') as f:
    current = None
    for line in f:
        line = line.strip()
        if line.startswith("id: "): current = line.split("id: ")[1]
        elif line.startswith("is_a: ") and current:
            p = line.split("is_a: ")[1].split(" ! ")[0]
            if current not in parents: parents[current] = set()
            parents[current].add(p)

print(">>> Propagating (Child <= Parent)...")
idx_to_parents = {}
for i, term in enumerate(top_terms):
    if term in parents:
        idx_to_parents[i] = [term_to_idx[p] for p in parents[term] if p in term_to_idx]

for i in tqdm(range(len(ensemble_preds)), desc="Propagating"):
    for child_idx, parent_indices in idx_to_parents.items():
        val = ensemble_preds[i, child_idx]
        for p_idx in parent_indices:
            if val > ensemble_preds[i, p_idx]:
                ensemble_preds[i, p_idx] = val

# ==========================================
# 5. SAVE
# ==========================================
print(">>> Saving submission_trinity.tsv...")
submission_lines = []
THRESHOLD = 0.01

for i in tqdm(range(len(test_ids)), desc="Writing"):
    pid = test_ids[i]
    indices = np.where(ensemble_preds[i] > THRESHOLD)[0]
    for idx in indices:
        submission_lines.append(f"{pid}\t{top_terms[idx]}\t{ensemble_preds[i][idx]:.3f}")

with open('submission_trinity.tsv', 'w') as f:
    f.write('\n'.join(submission_lines))

print(">>> SUCCESS! Download 'submission_trinity.tsv'")

Using device: cuda
>>> Loading Data...
>>> Building Target Matrix...


  0%|          | 0/342098 [00:00<?, ?it/s]


>>> Training Architecture: SimpleMLP...


Ep 1:   0%|          | 0/2576 [00:00<?, ?it/s]

Ep 2:   0%|          | 0/2576 [00:00<?, ?it/s]

...

>>> Diverse Ensemble Averaging Complete.
>>> Parsing Graph...
>>> Propagating (Child <= Parent)...


Propagating:   0%|          | 0/224309 [00:00<?, ?it/s]

>>> Saving submission_trinity.tsv...


Writing:   0%|          | 0/224309 [00:00<?, ?it/s]

>>> SUCCESS! Download 'submission_trinity.tsv'
